In [2]:
import glob
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.stats import norm

In [49]:
destination = '../../data/commaai/train_files_from_bag/'
bag_path = '../../data/commaai/train_bags_2/'
density_path = '../../data/commaai/density/fastkde_density.csv'
density = pd.read_csv(density_path)

In [4]:
an = pd.read_csv('../../data/commaai/train_files_from_bag/yaws.csv', header = None)

In [5]:
an.shape

(35212, 1)

In [ ]:
count = 0
nb_all = []
for i in range(0, len(all_files)):
    nb = len(all_files[i])
    print('bag ' +  str(i) +  ' has ' + str(nb) + ' observations')
    nb_all.append(nb)
    count += nb
print('overall number of samples: ' + str(count))

In [15]:
os.path.basename(all_files[2][10])

'129_209run1.png'

In [13]:
all_files[2]

['../../data/commaai/train_bags/3/2_195run1.png',
 '../../data/commaai/train_bags/3/3_195run1.png',
 '../../data/commaai/train_bags/3/4_195run1.png',
 '../../data/commaai/train_bags/3/24_195run1.png',
 '../../data/commaai/train_bags/3/25_195run1.png',
 '../../data/commaai/train_bags/3/165_195run1.png',
 '../../data/commaai/train_bags/3/166_195run1.png',
 '../../data/commaai/train_bags/3/196_195run1.png',
 '../../data/commaai/train_bags/3/127_209run1.png',
 '../../data/commaai/train_bags/3/128_209run1.png',
 '../../data/commaai/train_bags/3/129_209run1.png',
 '../../data/commaai/train_bags/3/130_209run1.png',
 '../../data/commaai/train_bags/3/131_209run1.png',
 '../../data/commaai/train_bags/3/98_248run1.png',
 '../../data/commaai/train_bags/3/107_248run1.png',
 '../../data/commaai/train_bags/3/116_248run1.png',
 '../../data/commaai/train_bags/3/117_248run1.png',
 '../../data/commaai/train_bags/3/125_248run1.png',
 '../../data/commaai/train_bags/3/126_248run1.png',
 '../../data/commaai/

In [ ]:
np.sum(nb_all[1:6])

In [ ]:
np.load('test_files_run2.npy')[0]

In [ ]:
def wrap_int64(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def wrap_bytes(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def wrap_float(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
def find_closest_element(y: float, arr: np.ndarray):
    index = np.searchsorted(arr,y)
    if (index >= 1) & (index < arr.shape[0]):
        res = [arr[index - 1], arr[index]]
    elif (index < arr.shape[0]):
        return np.array(index)
    else:
        return np.array(index - 1)

    if res[0] == res[1]:
        return np.array(index - 1)
    else:
        diff_pre = np.abs(y-res[0])
        diff_aft = np.abs(y-res[1])
        if diff_pre == diff_aft:
            return np.array(index - 1), 
        else:
            return index - 1 if diff_pre < diff_aft else index
def Fy(y, density, density_type = 'fast_kde' ):
    integral = density.loc[find_closest_element(y, density['axes']),'cdf']
    return(integral)  

In [ ]:
counts_minus_1 = np.zeros(20)
n = 0
while np.sum(counts) < 100000:
    
    #start new shard when more than 256 images have been written
    if (np.sum(counts_minus_1 - counts) > 256):
        n += 1
    
    out_path_shard = str(destination + str(n) + '_new2.tfrecords')

    print(out_path_shard)
            # start tfrecords writer

    with tf.io.TFRecordWriter(out_path_shard) as writer:

         for i in range(0,len(all_files)):

            if all_files[i]: 

                # draw new sample if bin is not full yet
                if counts[i] <= max_bins[i]:

                    # load random example from bag i
                    current_file = str(random.choice(all_files[i]))
                    x = np.load(current_file, allow_pickle=True)
                    img = x[0]
                    label = x[1]
                    tr_label = x[2]

                    # write to shard
                    img_bytes = img.tostring()
                    rows = img.shape[0]
                    cols = img.shape[1]
                    depth = img.shape[2]


                    # save image and label in dictionary
                    data = \
                            {
                                'image': wrap_bytes(img_bytes),
                                'label': wrap_float(label),
                                'rows': wrap_int64(rows),
                                'cols': wrap_int64(cols),
                                'depth': wrap_int64(depth),
                                'tr_label': wrap_float(tr_label)

                            }
                    feature = tf.train.Features(feature=data)

                    # Wrap again as a TensorFlow Example.
                    example = tf.train.Example(features=feature)

                    serialized = example.SerializeToString()


                    with open(str(destination + 'yaws.csv'), 'a') as csvfile:
                        csvfile.write("%s\n" % label)

                    # Write the serialized data to the TFRecords file.
                    writer.write(serialized)

                    counts[i] += 1

                    # mirror image
                    img = np.fliplr(img)
                    label = - label
                    tr_label = norm.ppf(Fy(label, density))

                    # write to shard
                    img_bytes = img.tostring()
                    rows = img.shape[0]
                    cols = img.shape[1]
                    depth = img.shape[2]


                    # save image and label in dictionary
                    data = \
                            {
                                'image': wrap_bytes(img_bytes),
                                'label': wrap_float(label),
                                'rows': wrap_int64(rows),
                                'cols': wrap_int64(cols),
                                'depth': wrap_int64(depth),
                                'tr_label': wrap_float(tr_label)

                            }
                    feature = tf.train.Features(feature=data)

                    # Wrap again as a TensorFlow Example.
                    example = tf.train.Example(features=feature)

                    serialized = example.SerializeToString()


                    with open(str(destination + 'yaws.csv'), 'a') as csvfile:
                        csvfile.write("%s\n" % label)

                    # Write the serialized data to the TFRecords file.
                    writer.write(serialized)

                    counts[i] += 1
            
            counts_minus_1 = counts.copy()


In [ ]:
max_bins = [10000, 10000, 10000, 7500, 5000, 2500, 1000, 1000, 1000, 1000,
           500, 500, 500, 500, 500, 500, 500, 500, 500, 500]

In [ ]:
current_file

In [ ]:
np.load(current_file, allow_pickle = True)

In [ ]:
len(max_bins)

In [ ]:
img = x[0]
angle = x[1]
tr_angle = x[2]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img/169)
print('angle: ' + str(angle))

In [ ]:
tr_angle